In [62]:
#%pip install selenium
# https://chromedriver.chromium.org/downloads

import time
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select

def preparar_driver(path_projeto):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option('prefs', 
        {
            "plugins.always_open_pdf_externally": True,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            'download.default_directory': path_projeto.replace('/', '\\') + '\\arquivos'
        }
    )
    driver = webdriver.Chrome(service=Service(f'{path_projeto}/chromedriver.exe'), options=chrome_options)
    driver.maximize_window()

    login_url = 'https://painelprc.trf5.jus.br/painelprc/auth'
    driver.get(login_url)

    time.sleep(5)
    driver.find_element(By.ID, 'btnAutenticar').click()

    time.sleep(3)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.find_element(By.ID, 'cardextratodemo').find_element(By.TAG_NAME, 'button').click()

    time.sleep(3)
    driver.find_element(By.ID, 'form:numeroRequisitorio').clear()
    return driver

def download_demonstrativo(driver, path_arquivos, requisitorio):

    print(f'Buscando requisitório nº {requisitorio}')

    nome_arquivo = f'{path_arquivos}/{requisitorio}.pdf'
    if os.path.isfile(nome_arquivo): return 'já existe'

    driver.find_element(By.ID, 'form:numeroRequisitorio').clear()
    driver.find_element(By.ID, 'form:numeroRequisitorio').send_keys(requisitorio)
    driver.find_element(By.CSS_SELECTOR, '#form\:idTipoRequisitorio > tbody > tr > td:nth-child(3) > div').click()

    time.sleep(1)
    driver.find_element(By.ID, 'form:j_idt65').click()
    time.sleep(3)

    tabela_resultados = driver.find_element(By.ID, 'outroForm:listaextratoXXT_data')
    qtd_celulas_tabela_resultado = len(tabela_resultados.find_elements(By.TAG_NAME, 'td'))
    if qtd_celulas_tabela_resultado == 1: return 'não encontrado'
    
    driver.find_element(By.ID, 'outroForm:listaextratoXXT:0:btnpdf').click()
    time.sleep(3)

    nome_padrao =  path_arquivos.replace("/", "\\") + '\\PRC'

    os.rename(nome_padrao, nome_arquivo)

    return 'baixado com sucesso'

In [60]:
path_projeto = 'C:/Projetos/Martorelli/Demonstrativos'
driver = preparar_driver(path_projeto)

In [63]:
requisitorios = [
    '2021.80.00.001.201851',
    '2021.80.00.001.200568',
    '2020.80.00.002.206149',
    '2020.80.00.002.207504',
    '2020.80.00.002.200562',
    '2021.80.00.002.204689',
    '2022.80.00.002.205747',
    '2021.80.00.001.203308',
    '2022.80.00.002.203927',
    '2022.80.00.002.203178'
]

path_arquivos = f'{path_projeto}/arquivos'

for requisitorio in requisitorios:
    print(f'{requisitorio} {download_demonstrativo(driver, path_arquivos, requisitorio)}')

Requisitório já baixado
Requisitório não encontrado
Requisitorio baixado com sucesso
Requisitorio baixado com sucesso
Requisitorio baixado com sucesso
Requisitorio baixado com sucesso
Requisitorio baixado com sucesso
Requisitorio baixado com sucesso
Requisitorio baixado com sucesso
Requisitorio baixado com sucesso


In [ ]:
print('Pasta dos arquivos:')
for file in os.scandir(f'{path_projeto}/arquivos'):
    print(file.name)